In [2]:
# https://www.kaggle.com/vkrahul/twitter-hate-speech

In [3]:
import pandas as pd
import os
import re
import spacy
from gensim.models.phrases import Phrases, Phraser
from time import time 
import multiprocessing
from gensim.models import Word2Vec
import bokeh.plotting as bp
from bokeh.models import HoverTool, BoxSelectTool
from bokeh.plotting import figure, show, output_notebook
from sklearn.manifold import TSNE
from sklearn.model_selection import train_test_split
import numpy as np
from sklearn.preprocessing import scale
import keras 
from tensorflow.keras.models import Sequential, Model 
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv1D, MaxPooling1D, Flatten, Dropout, Input, Embedding
from keras.layers.merge import Concatenate
from sklearn.feature_extraction.text import TfidfVectorizer
from wordcloud import WordCloud
from nltk.tokenize import RegexpTokenizer
from sklearn.metrics import confusion_matrix
import matplotlib as plt

Using TensorFlow backend.


In [4]:
df = pd.read_csv('train.csv')
del(df['id'])

In [5]:
df_clean = df
from nltk.tokenize import RegexpTokenizer
t = time()


tokenizer = RegexpTokenizer(r'\w+')
df_clean['clean'] = df_clean['tweet'].astype('str') 
df_clean.dtypes

df_clean["tokens"] = df_clean["clean"].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize everything: {} mins'.format(round((time() - t) / 60, 2)))
df_clean.head()

Time to tokenize everything: 0.01 mins


,label,tweet,clean,tokens
0,0,@user when a father is dysfunctional and is s...,@user when a father is dysfunctional and is s...,"[user, when, a, father, is, dysfunctional, and..."
1,0,@user @user thanks for #lyft credit i can't us...,@user @user thanks for #lyft credit i can't us...,"[user, user, thanks, for, lyft, credit, i, can..."
2,0,bihday your majesty,bihday your majesty,"[bihday, your, majesty]"
3,0,#model i love u take with u all the time in ...,#model i love u take with u all the time in ...,"[model, i, love, u, take, with, u, all, the, t..."
4,0,factsguide: society now #motivation,factsguide: society now #motivation,"[factsguide, society, now, motivation]"


In [6]:
import gensim
#w2v_model = gensim.models.KeyedVectors.load_word2vec_format('/Users/0x23/Desktop/*/Projects/GoogleNews-vectors-negative300.bin', binary = True)
w2v_model = gensim.models.KeyedVectors.load_word2vec_format('word2vec.bin', binary = True)
#m2.wv.most_similar(positive=["good"])


In [7]:
'''
#WORD2VEC()
cores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model
w2v_model = Word2Vec(min_count=20,
                     window=2,
                     size=300,
                     sample=6e-5, 
                     alpha=0.03, 
                     min_alpha=0.0007, 
                     negative=20,
                     workers=cores-1)

#BUILD_VOCAB()
t = time()
w2v_model.build_vocab(df_clean["tokens"], progress_per=1000)
print('Time to build vocab: {} mins'.format(round((time() - t) / 60, 2)))

#TRAIN()
w2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)
print('Time to train the model: {} mins'.format(round((time() - t) / 60, 2)))
'''

'\n#WORD2VEC()\ncores = multiprocessing.cpu_count() # Count the number of cores in a computer, important for a parameter of the model\nw2v_model = Word2Vec(min_count=20,\n                     window=2,\n                     size=300,\n                     sample=6e-5, \n                     alpha=0.03, \n                     min_alpha=0.0007, \n                     negative=20,\n                     workers=cores-1)\n\n#BUILD_VOCAB()\nt = time()\nw2v_model.build_vocab(df_clean["tokens"], progress_per=1000)\nprint(\'Time to build vocab: {} mins\'.format(round((time() - t) / 60, 2)))\n\n#TRAIN()\nw2v_model.train(df_clean["tokens"], total_examples=w2v_model.corpus_count, epochs=10000, report_delay=1)\nprint(\'Time to train the model: {} mins\'.format(round((time() - t) / 60, 2)))\n'

In [8]:
#words similar to movie
w2v_model.wv.most_similar(positive=["movie"])

#words similar to good
w2v_model.wv.most_similar(positive=["good"])

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  
/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  """


[('great', 0.2877359688282013),
 ('morning', 0.21683667600154877),
 ('bad', 0.19614830613136292),
 ('wednesday', 0.19148559868335724),
 ('sad', 0.18417347967624664),
 ('like', 0.1815328747034073),
 ('nice', 0.18066930770874023),
 ('coffee', 0.17778238654136658),
 ('bing', 0.16255837678909302),
 ('blessed', 0.15798832476139069)]

In [9]:
# w2v_model.save("word2vec.model")
# w2v_model.wv.save_word2vec_format('word2vec.bin', binary=True)

In [10]:
# defining the chart
output_notebook()
plot_tfidf = bp.figure(plot_width=700, plot_height=600, title="A map of 10000 word vectors",
    tools="pan,wheel_zoom,box_zoom,reset,hover,previewsave",
    x_axis_type=None, y_axis_type=None, min_border=1)

# getting a list of word vectors. limit to 10000. each is of 200 dimensions
word_vectors = [w2v_model[w] for w in list(w2v_model.wv.vocab.keys())[:5000]]

# dimensionality reduction. converting the vectors to 2d vectors
tsne_model = TSNE(n_components=2, verbose=1, random_state=0)
tsne_w2v = tsne_model.fit_transform(word_vectors)

# putting everything in a dataframe
tsne_df = pd.DataFrame(tsne_w2v, columns=['x', 'y'])
tsne_df['words'] = list(w2v_model.wv.vocab.keys())[:5000]

# plotting. the corresponding word appears when you hover on the data point.
plot_tfidf.scatter(x='x', y='y', source=tsne_df)
hover = plot_tfidf.select(dict(type=HoverTool))
hover.tooltips={"word": "@words"}
show(plot_tfidf)

Loading BokehJS ...

/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:8: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  


[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2043 samples in 0.020s...
[t-SNE] Computed neighbors for 2043 samples in 2.461s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2043
[t-SNE] Computed conditional probabilities for sample 2000 / 2043
[t-SNE] Computed conditional probabilities for sample 2043 / 2043
[t-SNE] Mean sigma: 6.150521
[t-SNE] KL divergence after 250 iterations with early exaggeration: 164.334290
[t-SNE] KL divergence after 1000 iterations: 3.317315


/Library/Frameworks/Python.framework/Versions/3.7/lib/python3.7/site-packages/ipykernel_launcher.py:16: DeprecationWarning: Call to deprecated `wv` (Attribute will be removed in 4.0.0, use self instead).
  app.launch_new_instance()


In [11]:
#First defining the X (input), and the y (output)
y = df['label'].values
X = np.array(df_clean["tokens"])

#And here is the train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
vectorizer = TfidfVectorizer(analyzer=lambda x: x, min_df=10)
matrix = vectorizer.fit_transform([x for x in X_train])
tfidf = dict(zip(vectorizer.get_feature_names(), vectorizer.idf_))
print ('vocab size :', len(tfidf))

vocab size : 2964


In [13]:
def buildWordVector(tokens, size):
    vec = np.zeros(size).reshape((1, size))
    count = 0.
    for word in tokens:
        try:
            vec += w2v_model[word].reshape((1, size)) * tfidf[word]
            count += 1.
        except KeyError: # handling the case where the token is not
                         # in the corpus. useful for testing.
            continue
    if count != 0:
        vec /= count
    return vec

In [14]:
train_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_train)])
train_vecs_w2v = scale(train_vecs_w2v)

test_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, X_test)])
test_vecs_w2v = scale(test_vecs_w2v)

print ('shape for training set : ',train_vecs_w2v.shape,'\nshape for test set : ', test_vecs_w2v.shape)

shape for training set :  (25569, 300) 
shape for test set :  (6393, 300)


In [15]:
model = Sequential()

model.add(Dense(128, activation='relu', input_dim=300))
model.add(Dropout(0.7))
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='adadelta',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 128)               38528     
_________________________________________________________________
dropout (Dropout)            (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 129       
Total params: 38,657
Trainable params: 38,657
Non-trainable params: 0
_________________________________________________________________


In [16]:
history = model.fit(train_vecs_w2v, y_train, epochs=20, batch_size=50,validation_data=(test_vecs_w2v,y_test))
loss, accuracy = model.evaluate(train_vecs_w2v, y_train, verbose=False)
print("Training Accuracy: {:.4f}".format(accuracy))
loss, accuracy = model.evaluate(test_vecs_w2v, y_test, verbose=False)
print("Testing Accuracy:  {:.4f}".format(accuracy))

Epoch 1/20
512/512 [==============================] - 1s 2ms/step - loss: 0.7627 - accuracy: 0.6338 - val_loss: 0.5198 - val_accuracy: 0.7813
Epoch 2/20
512/512 [==============================] - 1s 2ms/step - loss: 0.7582 - accuracy: 0.6351 - val_loss: 0.5086 - val_accuracy: 0.7935
Epoch 3/20
512/512 [==============================] - 1s 2ms/step - loss: 0.7377 - accuracy: 0.6452 - val_loss: 0.4976 - val_accuracy: 0.8034
Epoch 4/20
512/512 [==============================] - 1s 1ms/step - loss: 0.7283 - accuracy: 0.6498 - val_loss: 0.4872 - val_accuracy: 0.8123
Epoch 5/20
512/512 [==============================] - 1s 1ms/step - loss: 0.7094 - accuracy: 0.6625 - val_loss: 0.4770 - val_accuracy: 0.8226
Epoch 6/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6950 - accuracy: 0.6689 - val_loss: 0.4673 - val_accuracy: 0.8337
Epoch 7/20
512/512 [==============================] - 1s 1ms/step - loss: 0.6908 - accuracy: 0.6762 - val_loss: 0.4578 - val_accuracy: 0.8425
Epoch 

In [18]:
def plot_history(history):
    acc = history.history['acc']
    val_acc = history.history['val_acc']
    loss = history.history['loss']
    val_loss = history.history['val_loss']
    x = range(1, len(acc) + 1)

    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(x, acc, 'b', label='Training acc')
    plt.plot(x, val_acc, 'r', label='Validation acc')
    plt.title('Training and validation accuracy')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(x, loss, 'b', label='Training loss')
    plt.plot(x, val_loss, 'r', label='Validation loss')
    plt.title('Training and validation loss')
    plt.legend()
#plot_history(history)

In [58]:
all_words = [word for tokens in X for word in tokens]
all_sentence_lengths = [len(tokens) for tokens in X]
ALL_VOCAB = sorted(list(set(all_words)))
print("%s words total, with a vocabulary size of %s" % (len(all_words), len(ALL_VOCAB)))
print("Max sentence length is %s" % max(all_sentence_lengths))


####################### CHANGE THE PARAMETERS HERE #####################################
EMBEDDING_DIM = 300 # how big is each word vector
MAX_VOCAB_SIZE = 2964# how many unique words to use (i.e num rows in embedding vector)
MAX_SEQUENCE_LENGTH = 53 # max number of words in a comment to use

443676 words total, with a vocabulary size of 41490
Max sentence length is 59


In [20]:
model.save('model.h5')

**Import Test CSV and make predictions**

In [21]:
test_df = pd.read_csv('test.csv')
del(test_df['id'])
test_df

,tweet
0,#studiolife #aislife #requires #passion #dedic...
1,@user #white #supremacists want everyone to s...
2,safe ways to heal your #acne!! #altwaystohe...
3,is the hp and the cursed child book up for res...
4,"3rd #bihday to my amazing, hilarious #nephew..."
...,...
17192,thought factory: left-right polarisation! #tru...
17193,feeling like a mermaid ð #hairflip #neverre...
17194,#hillary #campaigned today in #ohio((omg)) &am...
17195,"happy, at work conference: right mindset leads..."


In [26]:
t = time()

test_df['clean'] = test_df['tweet'].astype('str') 
test_df.dtypes

test_df["tokens"] = test_df["clean"].apply(tokenizer.tokenize)
# delete Stop Words

print('Time to tokenize test cases: {} mins'.format(round((time() - t) / 60, 2)))
test_df.head()

Time to tokenize test cases: 0.0 mins


,tweet,clean,tokens
0,#studiolife #aislife #requires #passion #dedic...,#studiolife #aislife #requires #passion #dedic...,"[studiolife, aislife, requires, passion, dedic..."
1,@user #white #supremacists want everyone to s...,@user #white #supremacists want everyone to s...,"[user, white, supremacists, want, everyone, to..."
2,safe ways to heal your #acne!! #altwaystohe...,safe ways to heal your #acne!! #altwaystohe...,"[safe, ways, to, heal, your, acne, altwaystohe..."
3,is the hp and the cursed child book up for res...,is the hp and the cursed child book up for res...,"[is, the, hp, and, the, cursed, child, book, u..."
4,"3rd #bihday to my amazing, hilarious #nephew...","3rd #bihday to my amazing, hilarious #nephew...","[3rd, bihday, to, my, amazing, hilarious, neph..."


In [29]:
test_df_X = np.array(test_df["tokens"])

In [35]:
test_df_vecs_w2v = np.concatenate([buildWordVector(z, 300) for z in map(lambda x: x, test_df_X)])
test_df_vecs_w2v = scale(test_df_vecs_w2v)

In [41]:
temp = test_df_vecs_w2v[0]

In [ ]:
print ('shape for training set : ',train_vecs_w2v.shape,'\nshape for test set : ', test_vecs_w2v.shape, '\nshape for test df set : ', test_df_vecs_w2v.shape)

In [43]:
#model.predict(temp)